In [56]:
#on charge le dataset et on le met dans un dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

df = pd.read_csv('Data.csv', sep=',', low_memory=False)



In [57]:
# On ne peut pas garder toutes les colonnes, un choix s'opère donc. 

# Nous scindons les colonnes restantes dans plusieurs dataframes :

# Data, dataframe général, possédant "eventid","iyear","imonth","iday","extended","resolution","country_txt","region_txt","provstate","city","success", "suicide","attacktype1_txt", "targtype1_txt","targsubtype1_txt","natlty1_txt","gname","nperps","claimed","weaptype1_txt","weapsubtype1_txt","nkill","nwound","dbsource"
Data = df[["eventid","iyear","imonth","iday","extended","resolution","country_txt","region_txt","provstate","city","success", "suicide","attacktype1_txt", "targtype1_txt","targsubtype1_txt","natlty1_txt","gname","nperps","claimed","weaptype1_txt","weapsubtype1_txt","nkill","nwound","dbsource"]]



In [58]:
# geo, dataframe ayant pour but la géolocalisation, comprenant "eventid", "latitude", "longitude","region_txt","country_txt"
geo = df[["eventid", "latitude", "longitude","region_txt","country_txt"]].dropna()

geo.to_csv("Geo.csv")

In [59]:
#headlines, dataframe comprenant les descriptions. "eventid","summary"
headlines = df[["eventid","summary"]]

On va tester les données géographiques avec le dataframe geo

In [60]:
geo.describe()

,eventid,latitude,longitude
count,1.771340e+05,177134.000000,1.771340e+05
mean,2.002979e+11,23.498444,-4.586957e+02
std,1.321102e+09,18.569247,2.047790e+05
min,1.970000e+11,-53.154613,-8.618590e+07
25%,1.991061e+11,11.510545,4.545640e+00
50%,2.009070e+11,31.467463,4.324651e+01
75%,2.014083e+11,34.685087,6.871033e+01
max,2.017123e+11,74.633553,1.793667e+02


In [78]:
import folium
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

# créer la carte centrée sur la position moyenne des événements
map = folium.Map(location=[geo['latitude'].mean(), geo['longitude'].mean()], zoom_start=4)

# créer un groupe de marqueurs clusterisés
marker_cluster = MarkerCluster().add_to(map)

# ajouter un marqueur pour chaque événement dans le groupe de marqueurs clusterisés
for index, row in geo.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(marker_cluster)

# créer un groupe pour afficher le nombre de marqueurs dans chaque cluster
sub_group = folium.plugins.FeatureGroupSubGroup(marker_cluster, 'Events Count')

# ajouter le groupe à la carte
map.add_child(sub_group)

# ajouter un compteur pour chaque cluster
for i, layer in enumerate(marker_cluster._children.values()):
    count = len(layer._children)
    bounds = layer.get_bounds()
    pos = [(bounds[0][0] + bounds[1][0]) / 2, (bounds[0][1] + bounds[1][1]) / 2]
    folium.Marker(location=pos, icon=folium.Icon(icon='info-sign'), tooltip=str(count)).add_to(sub_group)

map.save("carte.html")

KeyboardInterrupt: 

In [63]:
# On regarde le nombre de d'attentats revendiqués par rapport au nombre total 

Revendiqué = df[["claimed"]]

#On récupère le nombre de ligne ou la colonne "claimed" est égale à 1 dans la variable "count"

count = Revendiqué[Revendiqué.claimed == 1].count()

#On récupère le nombre de ligne total dans la variable "count2"

count2 = Revendiqué.count()

#On calcule le pourcentage de ligne ou la colonne "claimed" est égale à 1

pourcentage = (count/count2)*100

print(pourcentage)


claimed    16.445302
dtype: float64


Test de séparation par continents :

In [77]:
# Créer un dataframe pour comptabiliser le nombre d'événements par pays et continent
events_by_country = geo.groupby(['region_txt', 'country_txt']).size().reset_index(name='count')

# Boucle pour créer une carte pour chaque continent
for continent in events_by_country['region_txt'].unique():

    # Créer un dataframe pour le continent en cours
    continent_df = events_by_country[events_by_country['region_txt'] == continent]

   # Obtenir les informations de localisation pour chaque pays dans le continent
    country_locations = geo.loc[geo['region_txt'] == continent, ['latitude', 'longitude', 'country_txt']].drop_duplicates()

    # Créer une carte centrée sur la position moyenne des événements pour ce continent
    map = folium.Map(location=[country_locations['latitude'].mean(), country_locations['longitude'].mean()], zoom_start=4)

    # Ajouter un marqueur pour chaque pays dans le continent
    for index, row in country_locations.iterrows():
     folium.Marker(location=[row['latitude'], row['longitude']], popup=row['country_txt'] + ': ' + str(events_by_country[(events_by_country['region_txt'] == continent) & (events_by_country['country_txt'] == row['country_txt'])]['count'].values[0])).add_to(map)

    # Enregistrer la carte
    map.save(continent + '_events_map.html')
